In [74]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns',23)
arquivo = pd.read_csv('D:/Github/data-science/machine-learning/algoritmos/exercicio-pratico-final/recipeData.csv')

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe8 in position 9775: invalid continuation byte

In [ ]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns',23)
arquivo = pd.read_csv('D:/Github/data-science/machine-learning/algoritmos/exercicio-pratico-final/recipeData.csv', encoding = "ISO-8859-1")

In [ ]:
arquivo.head(3)

In [ ]:
arquivo.shape

In [ ]:
faltantes = arquivo.isnull().sum()
faltantes_percentual = (arquivo.isnull().sum() / len(arquivo['StyleID'])) * 100
print(faltantes_percentual)

In [ ]:
arquivo['StyleID'].value_counts()

In [ ]:
selecao = arquivo.loc[arquivo['StyleID'].isin([7, 10, 134, 9, 4, 30, 86, 12, 92, 6, 175, 39])]

In [ ]:
selecao.shape

In [ ]:
selecao.drop('BeerID', axis = 1, inplace = True)
selecao.drop('Name', axis = 1, inplace = True)
selecao.drop('URL', axis = 1, inplace = True)
selecao.drop('Style', axis = 1, inplace = True)
selecao.drop('UserId', axis = 1, inplace = True)
selecao.drop('PrimingMethod', axis = 1, inplace = True)
selecao.drop('PrimingAmount', axis = 1, inplace = True)

In [ ]:
selecao.head(3)

In [ ]:
selecao.dtypes

In [ ]:
arquivo['SugarScale'].value_counts()

In [ ]:
arquivo['BrewMethod'].value_counts()

In [ ]:
selecao['SugarScale'] = selecao['SugarScale'].replace('Specific Gravity', 0)
selecao['SugarScale'] = selecao['SugarScale'].replace('Plato', 1)

In [ ]:
selecao.head(3)

In [ ]:
#Transformando variáveis texto na coluna 'BreawMethod' em categorias com one hot enconding
brewmethod_encode = pd.get_dummies(selecao['BrewMethod'])

#Excluindo a coluna de texto 'BrewMethod'
selecao.drop('BrewMethod', axis = 1, inplace = True)

#Inserindo as variaveis one hot encode novamente no dataset
concatenado = pd.concat([selecao, brewmethod_encode], axis=1)

In [ ]:
concatenado.head(3)

In [ ]:
faltantes = concatenado.isnull().sum()
faltantes_percentual = (concatenado.isnull().sum()) / len(concatenado['StyleID']) * 100
print(faltantes_percentual)

In [ ]:
import matplotlib.pyplot as plt
concatenado.boxplot(column= ['BoilGravity', 'MashThickness', 'PitchRate', 'PrimaryTemp'])
plt.show()

In [ ]:
concatenado.hist(column= ['BoilGravity', 'MashThickness', 'PitchRate', 'PrimaryTemp'])
plt.show()

In [ ]:
concatenado['PitchRate'].fillna(concatenado['PitchRate'].mean(), inplace=True)
concatenado.fillna(concatenado.median(), inplace=True)

In [ ]:
faltantes = concatenado.isnull().sum()
faltantes_percentual = (concatenado.isnull().sum() / len(concatenado['StyleID'])) * 100
print(faltantes_percentual)

In [ ]:
#Definindo variáveis preditoras e variável target
y = concatenado['StyleID']
x = concatenado.drop('StyleID', axis=1)

In [ ]:
def modeloclassificacao(a,b):
    from sklearn.linear_model import LogisticRegression
    from sklearn.naive_bayes import GaussianNB
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.neighbors import KNeighborsClassifier #KNN precisa de normalização
    from sklearn.preprocessing import MinMaxScaler
    from sklearn.model_selection import StratifiedKFold
    from sklearn.model_selection import cross_val_score
    skfold = StratifiedKFold(n_splits=3)
    x = a
    y = b
    #Normalizando as variáveis preditoras para o KNN:
    normalizador = MinMaxScaler(feature_range = (0, 1))
    x_norm = normalizador.fit_transform(x)
    
    logist = LogisticRegression()
    naive = GaussianNB()
    decision_tree = DecisionTreeClassifier()
    knn = KNeighborsClassifier()
    resul_logist = cross_val_score(logist,x,y,cv = skfold)
    resul_naive = cross_val_score(naive,x,y,cv = skfold)
    resul_decision_tree = cross_val_score(decision_tree,x,y,cv = skfold)
    resul_knn = cross_val_score(knn,x_norm,y,cv= skfold)#Recebe a variável normalizada
    dic_classmodels = {'Logistica' :resul_logist.mean(), 'Naive' :resul_naive.mean(), 'Decision Tree' :resul_decision_tree.mean(), 'KNN' :resul_knn.mean()}
    melhor_modelo = max(dic_classmodels, key=dic_classmodels.get)
    print('Reg. Logística:',resul_logist.mean(), '|Naive Bayes:',resul_naive.mean(), '|Decision Tree:',resul_decision_tree.mean(), '|KNN:',resul_knn.mean())
    print('O melhor modelo foi:', melhor_modelo, 'com o valor:', dic_classmodels[melhor_modelo])

In [ ]:
modeloclassificacao(x,y)

In [76]:
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV

#Normalizando as variáveis preditoras:
normalizador = MinMaxScaler(feature_range = (0 , 1))
X_norm = normalizador.fit_transform(x)

#Definindo os valores que serão testados no KNN:
valores_K = np.array([3,5,7,9])
calculo_distancia = ['minkowski', 'chebyshev']
valores_p = np.array([1,2,3])
valores_grid = {'n_neighbors':valores_K, 'metric':calculo_distancia, 'p':valores_p}

#Criação do modelo:
modelo = KNeighborsClassifier()

#Criando os grids:
gridKNN = GridSearchCV(estimator = modelo, param_grid = valores_grid, cv=3, n_jobs=-1)
gridKNN.fit(X_norm,y)

#Imprimindo os melhores parâmetros:
print("Melhor acurácia: ", gridKNN.best_score_)
print("Melhor K: ", gridKNN.best_estimator_.n_neighbors)
print("Método distância: ", gridKNN.best_estimator_.metric)
print("Melhor valor p: ", gridKNN.best_estimator_.p)

Melhor acurácia:  0.5072549683830171
Melhor K:  9
Método distância:  minkowski
Melhor valor p:  1
